In [1]:
import sys
sys.path.append('../')

In [2]:
import warnings
import logging
import time
import psutil
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime

from its_logging.logger_config import logger
from utils.its_utils import clip_to_california, get_wfr_tf_template
from utils.gdf_utils import repair_geometries, show_columns, verify_gdf_columns
from utils.add_common_columns import add_common_columns
from utils.enrich_polygons import enrich_polygons
from utils.keep_fields import keep_fields
from utils.assign_domains import assign_domains
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [3]:
a_reference_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
start_year, end_year = 2020, 2025

process = psutil.Process(os.getpid())

In [4]:
from enrich.enrich_NPS import enrich_NPS_from_gdb

In [5]:
nps_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\NPS_2023\New_NPS_2023_20240625_ReisThomasViaUpload_1.gdb'
nps_layer_name = 'NPS_2023_20240625_ReisThomasViaUpload2'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\NPS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"NPS_enriched_{datetime.today().strftime('%Y%m%d')}"



In [6]:
NPS_gpd = enrich_NPS_from_gdb(nps_gdb_path,
                    nps_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-02-05 03:01:21,961 INFO  [enrich.enrich_NPS     ]  Load the NPS data into a GeoDataFrame
2025-02-05 03:01:22,542 INFO  [enrich.enrich_NPS     ]     time for loading NPS_2023_20240625_ReisThomasViaUpload2: 0.5802760124206543
2025-02-05 03:01:22,543 INFO  [enrich.enrich_NPS     ]     all required columns are present.
2025-02-05 03:01:22,986 INFO  [enrich.enrich_NPS     ]  Performing Standardization...
2025-02-05 03:01:22,987 INFO  [enrich.enrich_NPS     ]     step 1/11 select after 1995
2025-02-05 03:01:23,000 INFO  [enrich.enrich_NPS     ]     step 2/11 repairing geometry
2025-02-05 03:01:26,437 INFO  [enrich.enrich_NPS     ]     step 3/11 clip features by CA
2025-02-05 03:01:36,067 INFO  [enrich.enrich_NPS     ]     step 4/11 dissolve to implement multipart polygons
2025-02-05 03:01:36,695 INFO  [enrich.enrich_NPS     ]     step 5/11 rename and add fields
2025-02-05 03:01:36,719 INFO  [enrich.enrich_NPS     ]     step 6/11 import attributes
2025-02-05 03:01:36,742 INFO  [enrich.en

2025-02-05 03:11:19,438 INFO  [utils.counts_to_mas   ]              counts step 8/8: set to 'NO' if Org is 'USFS' and Status is 'Active'
2025-02-05 03:11:19,441 INFO  [utils.enrich_polygons ]           Crosswalk Complete. Continuing Enrichment...
2025-02-05 03:11:19,442 INFO  [utils.enrich_polygons ]              enrich step 29/32 Calculating Latitude and Longitude...
2025-02-05 03:11:19,455 INFO  [utils.enrich_polygons ]              enrich step 30/32 calculate treatment acres
2025-02-05 03:11:19,457 INFO  [utils.enrich_polygons ]              enrich step 31/32 removing unnecessary fields
2025-02-05 03:11:19,460 INFO  [utils.enrich_polygons ]              enrich step 32/32 delete if County is Null
2025-02-05 03:11:19,713 INFO  [enrich.enrich_NPS     ]     step 9/11 adding treatment ID
2025-02-05 03:11:19,717 INFO  [enrich.enrich_NPS     ]     step 10/11 Assign Domains...
2025-02-05 03:11:20,018 INFO  [utils.assign_domains  ]        Created domain 'D_OBJECTIVE' with 27 values
2025-02-0

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\NPS_2020_2025.gdb


2025-02-05 03:11:20,776 INFO  [its_logging.logger_config]  Memory usage: 364.01 MB


In [7]:
NPS_reference = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', driver='openFileGDB', layer='nps_flat_fuels_enriched2023_20240925')

In [8]:
NPS_reference[['ACTIVITY_QUANTITY', 'ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]].groupby(['ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]).sum()

ACTIVITY_QUANTITY
ADMINISTERING_ORG COUNTS_TO_MAS PRIMARY_OBJECTIVE     Year_txt                      
NPS               NO            ECO_RESTOR            2023       481.000000000000000
                                NOT_DEFINED           2021      1018.000000000000000
                  YES           ECO_RESTOR            2020      1115.000000000000000
                                                      2021       451.000000000000000
                                                      2023       246.000000000000000
                                OTHER_FUELS_REDUCTION 2020      1050.000000000000000
                                                      2021      3743.000000000000000
                                                      2022      3609.000000000000000
                                                      2023      4508.000000000000000
                                PRESCRB_FIRE          2020     12051.000000000000000
                                                      2021      3480.000000000000000

In [9]:
NPS_output = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)

In [10]:
NPS_output[['ACTIVITY_QUANTITY', 'ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]].groupby(['ADMINISTERING_ORG', 'COUNTS_TO_MAS', 'PRIMARY_OBJECTIVE', "Year_txt"]).sum()

ACTIVITY_QUANTITY
ADMINISTERING_ORG COUNTS_TO_MAS PRIMARY_OBJECTIVE     Year_txt                      
NPS               NO            ECO_RESTOR            2023       481.000000000000000
                                NOT_DEFINED           2021      1018.000000000000000
                  YES           ECO_RESTOR            2020      1115.000000000000000
                                                      2021       451.000000000000000
                                                      2023       246.000000000000000
                                OTHER_FUELS_REDUCTION 2020      1050.000000000000000
                                                      2021      3743.000000000000000
                                                      2022      3609.000000000000000
                                                      2023      4508.000000000000000
                                PRESCRB_FIRE          2020     12051.000000000000000
                                                      2021      3480.000000000000000

In [8]:
from enrich.enrich_BLM import enrich_BLM

In [9]:
blm_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\BLM_2023\BLM_2010_2023_fromReisThomasViaUpload.gdb"
blm_input_layer_name = "BLM_2010_2023_fromReisThomasViaUpload"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"BLM_enriched_{datetime.today().strftime('%Y%m%d')}"

In [11]:
from enrich.enrich_CNRA import enrich_CNRA

In [12]:

cnra_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CNRA_2023\CNRA_Tracker_Data_Export_20241120.gdb"
cnra_polygon_layer_name = "TREATMENT_POLY"
cnra_line_layer_name = "TREATMENT_LINE"
cnra_point_layer_name = "TREATMENT_POINT"
cnra_project_polygon_layer_name = "PROJECT_POLY"
cnra_activity_layer_name = "ACTIVITIES"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CNRA_enriched_{datetime.today().strftime('%Y%m%d')}"


In [ ]:

enrich_CNRA(cnra_input_gdb_path,
            cnra_polygon_layer_name,
            cnra_line_layer_name,
            cnra_point_layer_name,
            cnra_project_polygon_layer_name,
            cnra_activity_layer_name,
            a_reference_gdb_path,
            start_year,
            end_year,
            output_gdb_path,
            output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")


2025-02-05 03:11:52,774 INFO  [enrich.enrich_CNRA    ]  Load the CNRA polygon layer into a GeoDataFrame
2025-02-05 03:11:53,283 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-05 03:11:53,971 INFO  [enrich.enrich_CNRA    ]  Load the CNRA line layer into a GeoDataFrame
2025-02-05 03:11:53,987 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-05 03:11:53,990 INFO  [enrich.enrich_CNRA    ]  Load the CNRA point layer into a GeoDataFrame
2025-02-05 03:11:54,008 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-05 03:11:54,010 INFO  [enrich.enrich_CNRA    ]  Load the CNRA project polygon layer into a GeoDataFrame
2025-02-05 03:11:54,222 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-05 03:11:54,628 INFO  [enrich.enrich_CNRA    ]  Load the CNRA activity layer into a DataFrame
2025-02-05 03:11:55,401 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-05 03:

2025-02-05 03:25:10,891 INFO  [utils.enrich_polygons ]              enrich step 28/32 Initiating Crosswalk...
2025-02-05 03:25:10,892 INFO  [utils.crosswalk       ]           Calculating Crosswalking Activites...
2025-02-05 03:25:10,892 INFO  [utils.crosswalk       ]              Load Crosswalk table...
2025-02-05 03:25:10,956 INFO  [utils.crosswalk       ]              cross step 1/8 add join
2025-02-05 03:25:11,024 INFO  [utils.crosswalk       ]              cross step 2/8 calculate activities
2025-02-05 03:25:11,028 INFO  [utils.crosswalk       ]              cross step 3/8 calculate residue fate field
2025-02-05 03:25:11,029 INFO  [utils.crosswalk       ]              cross step 4/8 select attribute by layer
2025-02-05 03:25:11,032 INFO  [utils.crosswalk       ]              cross step 5/8 calculating objective...
2025-02-05 03:25:11,059 INFO  [utils.crosswalk       ]              cross step 6/8 calculate category
2025-02-05 03:25:11,353 INFO  [utils.crosswalk       ]              

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb


2025-02-05 03:25:20,709 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb CNRA_enriched_20250205_polygon
2025-02-05 03:25:20,866 INFO  [enrich.enrich_CNRA    ]  Enrich the CNRA lines...
2025-02-05 03:25:20,866 INFO  [enrich.enrich_CNRA    ]     Part 1 Prepare Features
2025-02-05 03:25:20,869 INFO  [enrich.enrich_CNRA    ]     Part 2 Prepare Activity Table
2025-02-05 03:25:21,453 INFO  [enrich.enrich_CNRA    ]        step 2/17 remove milliseconds from dates
2025-02-05 03:25:21,475 INFO  [enrich.enrich_CNRA    ]        step 3/17 create standardized activity table
2025-02-05 03:25:21,480 INFO  [enrich.enrich_CNRA    ]        step 4/17 import activities into standardized table
2025-02-05 03:25:21,509 INFO  [enrich.enrich_CNRA    ]        step 5/17 calculate unique Treatment ID with postfix '-CNRA'
2025-02-05 03:25:21,521 INFO  [enrich.enrich_CNRA    ]     Part 3 - Combine CNRA Features and Activity Tabl

2025-02-05 03:25:35,444 INFO  [utils.assign_domains  ]        Created domain 'D_CNTY' with 59 values
2025-02-05 03:25:35,446 INFO  [utils.assign_domains  ]        Created domain 'D_IN_WUI' with 4 values
2025-02-05 03:25:35,451 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVDSCRP' with 61 values
2025-02-05 03:25:35,454 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVCAT' with 8 values
2025-02-05 03:25:35,455 INFO  [utils.assign_domains  ]        Created domain 'D_USERDEFINED' with 2 values
2025-02-05 03:25:35,458 INFO  [utils.assign_domains  ]        Created domain 'D_BVT' with 8 values
2025-02-05 03:25:35,460 INFO  [utils.assign_domains  ]        Created domain 'D_RESIDUEFATE' with 15 values
2025-02-05 03:25:35,462 INFO  [utils.assign_domains  ]        Created domain 'D_UOM' with 6 values
2025-02-05 03:25:35,464 INFO  [utils.assign_domains  ]        Created domain 'D_TASKFORCE' with 5 values
2025-02-05 03:25:35,466 INFO  [utils.assign_domains  ]        Created 

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb


2025-02-05 03:25:36,221 INFO  [enrich.enrich_CNRA    ]        step 2/17 remove milliseconds from dates
2025-02-05 03:25:36,237 INFO  [enrich.enrich_CNRA    ]        step 3/17 create standardized activity table
2025-02-05 03:25:36,241 INFO  [enrich.enrich_CNRA    ]        step 4/17 import activities into standardized table
2025-02-05 03:25:36,267 INFO  [enrich.enrich_CNRA    ]        step 5/17 calculate unique Treatment ID with postfix '-CNRA'
2025-02-05 03:25:36,279 INFO  [enrich.enrich_CNRA    ]     Part 3 - Combine CNRA Features and Activity Table
2025-02-05 03:25:36,280 INFO  [enrich.enrich_CNRA    ]        step 6/17 join polygon table and activity table
2025-02-05 03:25:36,393 INFO  [enrich.enrich_CNRA    ]     Part 4 Prepare Project Table
2025-02-05 03:25:36,395 INFO  [enrich.enrich_CNRA    ]        step 7/17 calculate unique Project ID if null
2025-02-05 03:25:36,422 INFO  [enrich.enrich_CNRA    ]     Part 5 Join Project Table to Features/Activities
2025-02-05 03:25:36,472 INFO  

2025-02-05 03:29:06,109 INFO  [utils.assign_domains  ]        Created domain 'D_FNDSRC' with 11 values
2025-02-05 03:29:06,111 INFO  [utils.assign_domains  ]        Created domain 'D_AGENCY' with 8 values
2025-02-05 03:29:06,114 INFO  [utils.assign_domains  ]        Created domain 'D_ORGANIZATION' with 31 values
2025-02-05 03:29:06,116 INFO  [utils.assign_domains  ]        Warning: 1 rows with NULL values were dropped from 'D_DATASTATUS'
2025-02-05 03:29:06,116 INFO  [utils.assign_domains  ]        Created domain 'D_DATASTATUS' with 2 values
2025-02-05 03:29:06,119 INFO  [utils.assign_domains  ]        Created domain 'D_DATAMSG' with 3 values
2025-02-05 03:29:06,121 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-02-05 03:29:06,123 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-02-05 03:29:06,124 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-02-05 03:29:06,145 INFO  [uti

In [11]:
# temp file path
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CALTRANS_2023\Caltrans_Vegetation_Management_20_23.gdb"
tree_activity_layer_name = "Caltrans_Vegetation_Management_Trees_ActivitiesTable_20_23"
tree_treatment_layer_name = "Caltrans_Vegetation_Management_Trees_Treatments_20_23"
road_activity_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_20_23"
road_treatment_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_20_23"
start_year = 2021
end_year = 2023
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"